In [ ]:
nums = list(range(0, 1000001))

# parallelize
nums_rdd = sc.parallelize(nums) # this distributes the data and returns an RDD to nums_rdd

# collect
nums_rdd.collect() 

# take
nums_rdd.take(5)

# map
squared_nums_rdd = nums_rdd.map(lambda x: x ** 2)
pairs = nums_rdd.map(lambda x: (x, len(str(x))))
pairs.take(25) # will show us that this worked

# filter
even_pairs = pairs.filter(lambda x: (x[1] % 2) == 0)
even_pairs.take(25)

# groupByKey
swapped_pairs = even_pairs.map(lambda x: (x[1], x[0])) # make the number of digits the key for easier grouping
grouped = swapped_pairs.groupByKey()
grouped = grouped.map(lambda x: (x[0], list(x[1]))) # makes the output look nicer
grouped.take(25)
averaged = grouped.map(lambda x: (x[0], sum(x[1]) / len(x[1])))
averaged.collect() # can collect this since we only have a few

